# 12.Convolutinal Neural Network using KerasAPI

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [23]:
# Hyper parameters
num_epochs = 10
num_classes = 10
batch_size = 32
learning_rate = 0.001

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

print("training_data\n", x_train.shape)
print("test_data\n", x_test.shape)
print("training_label\n", y_train.shape)
print("test_label\n", y_test.shape)

training_data
 (50000, 32, 32, 3)
test_data
 (10000, 32, 32, 3)
training_label
 (50000, 1)
test_label
 (10000, 1)


In [24]:
x_train_ = tf.transpose(tf.convert_to_tensor(x_train, dtype=tf.float32), 
                        [0, 3, 1, 2])
y_train_ = tf.reshape(tf.one_hot(y_train, 10), (-1, 10))


print(x_train_.shape)
print(y_train_.shape)

(50000, 3, 32, 32)
(50000, 10)


In [25]:
train_dataset = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .batch(batch_size)
    .shuffle(10000)
)

train_dataset = (
    train_dataset.map(lambda x, y: 
                      (tf.math.divide(tf.cast(
                          tf.transpose(x, [0, 3, 1, 2]), tf.float32), 255.0), 
                       tf.reshape(tf.one_hot(y, 10), (-1, 10))))
)

train_dataset = train_dataset.repeat()
print(train_dataset)

<RepeatDataset shapes: ((None, 3, 32, 32), (None, 10)), types: (tf.float32, tf.float32)>


In [26]:
test_dataset = (
    tf.data.Dataset.from_tensor_slices((x_test, y_test))
    .batch(1000)
    .shuffle(10000)
)
test_dataset = (
    test_dataset.map(lambda x, y: 
                      (tf.math.divide(tf.cast(
                          tf.transpose(x, [0, 3, 1, 2]), tf.float32), 255.0), 
                       tf.reshape(tf.one_hot(y, 10), (-1, 10))))
)
test_dataset = test_dataset.repeat()
print(test_dataset)

<RepeatDataset shapes: ((None, 3, 32, 32), (None, 10)), types: (tf.float32, tf.float32)>


In [27]:
class Cifar10Model(tf.keras.Model):
    def __init__(self):
        super(Cifar10Model, self).__init__(name='cifar_cnn')
        
        self.conv_block1 = tf.keras.Sequential([
            L.Conv2D(
                8, 
                5,
                padding='same',
                activation=tf.nn.relu,
                kernel_initializer=tf.keras.initializers.VarianceScaling(),
                kernel_regularizer=tf.keras.regularizers.l2(l=0.001),
                data_format="channels_first"
            ),
            L.MaxPooling2D(
                (3, 3), 
                (2, 2), 
                padding='same',
                data_format="channels_first"
            ),
            L.BatchNormalization(axis=1),
        ])

        self.conv_block2 = tf.keras.Sequential([
            L.Conv2D(
                16, 
                5,
                padding='same',
                activation=tf.nn.relu,
                kernel_initializer=tf.keras.initializers.VarianceScaling(),
                kernel_regularizer=tf.keras.regularizers.l2(l=0.001),
                data_format="channels_first"
            ),
            L.MaxPooling2D(
                (3, 3), 
                (2, 2), 
                padding='same',
                data_format="channels_first"
            ),
            L.BatchNormalization(axis=1),
        ])
        
        self.conv_block3 = tf.keras.Sequential([
            L.Conv2D(
                32, 
                5,
                padding='same',
                activation=tf.nn.relu,
                kernel_initializer=tf.keras.initializers.VarianceScaling(),
                kernel_regularizer=tf.keras.regularizers.l2(l=0.001),
                data_format="channels_first"
            ),
            L.MaxPooling2D(
                (3, 3), 
                (2, 2), 
                padding='same',
                data_format="channels_first"
            ),
            L.BatchNormalization(axis=1),
        ])
        
        self.flatten = L.Flatten()
        self.fc1 = L.Dense(
            256, 
            activation=tf.nn.relu,
            kernel_initializer=tf.keras.initializers.VarianceScaling(),
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001))
        self.dropout = L.Dropout(rate=0.8)
        self.fc2 = L.Dense(10)
        self.softmax = L.Softmax()

    def call(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.flatten(x)
        x = self.dropout(self.fc1(x))
        x = self.fc2(x)
        return self.softmax(x)

In [28]:
model = Cifar10Model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
y_init = model(x_train_[:100])
y_init.shape

TensorShape([100, 10])

In [29]:
model.summary()

Model: "cifar_cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_9 (Sequential)    multiple                  640       
_________________________________________________________________
sequential_10 (Sequential)   multiple                  3280      
_________________________________________________________________
sequential_11 (Sequential)   multiple                  12960     
_________________________________________________________________
flatten_3 (Flatten)          multiple                  0         
_________________________________________________________________
dense_6 (Dense)              multiple                  131328    
_________________________________________________________________
dropout_3 (Dropout)          multiple                  0         
_________________________________________________________________
dense_7 (Dense)              multiple                  25

In [31]:
steps_per_epoch = int(x_train.shape[0]/batch_size)

model.fit(
    train_dataset,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    verbose=1,
    shuffle=True,
    workers=5
)

Train for 1562 steps
Epoch 1/10
1562/1562 [==============================] - 8s 5ms/step - loss: 2.1312 - accuracy: 0.3173
Epoch 2/10
1562/1562 [==============================] - 7s 4ms/step - loss: 1.7109 - accuracy: 0.4451
Epoch 3/10
1562/1562 [==============================] - 7s 5ms/step - loss: 1.5162 - accuracy: 0.5227
Epoch 4/10
1562/1562 [==============================] - 7s 5ms/step - loss: 1.4080 - accuracy: 0.5708
Epoch 5/10
1562/1562 [==============================] - 7s 4ms/step - loss: 1.3424 - accuracy: 0.5992
Epoch 6/10
1562/1562 [==============================] - 7s 5ms/step - loss: 1.3027 - accuracy: 0.6153
Epoch 7/10
1562/1562 [==============================] - 7s 5ms/step - loss: 1.2747 - accuracy: 0.6342
Epoch 8/10
1562/1562 [==============================] - 7s 5ms/step - loss: 1.2453 - accuracy: 0.6501
Epoch 9/10
1562/1562 [==============================] - 7s 5ms/step - loss: 1.2249 - accuracy: 0.6602
Epoch 10/10
1562/1562 [==============================] - 7s 5

In [32]:
train_loss, train_acc = model.evaluate(train_dataset, 
                                       steps=int(y_train.shape[0]/batch_size))
print("train_acc: {:0.3f}".format(train_acc))

test_loss, test_acc = model.evaluate(test_dataset, 
                                     steps=int(y_test.shape[0]/1000))
print("test_acc: {:0.3f}".format(test_acc))

1562/1562 [==============================] - 5s 3ms/step - loss: 1.0771 - accuracy: 0.7102
train_acc: 0.710
10/10 [==============================] - 0s 26ms/step - loss: 1.1741 - accuracy: 0.6766
test_acc: 0.677
